<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4c6ef3958f503f64837718cb3d911a9eca8220759b32a89895298067601b705a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 12:10:57
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.40 C
-------------------
Today PnL: 75.65 K (0.54%)
Current PnL: -17.28 L (-11.62%)
CY Booked + Current PnL: -8.24 L (-5.54%)
-------------------
Total profit:  3.62 L
Total loss:  -20.89 L
-------------------
Total Booked + Current PnL: 18.74 L (15.4%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.46%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.95 L (56.41%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-14.95,67.0,H-LC,8.58,117104.0,12792.0,3853.0,1.52,12.26,3.29,15.96,9.0,3.32,0.85,31.23,X5K,ATH,METALS
53,MEDANTA,1486.00,17.65,67.0,H-MC,15.71,167968.0,41768.0,4401.0,4.14,33.10,2.62,36.59,108.0,9.49,1.22,49.81,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.13,69.0,H-MC,6.82,141816.0,26880.0,9700.0,0.99,23.39,6.84,31.83,81.0,2.77,1.03,47.00,X40N,ATH,IT
82,TTKPRESTIG,770.00,106.62,69.0,M-SC,6.98,89060.0,-11717.0,11809.0,-0.31,-11.63,13.26,0.09,245.0,-0.99,0.65,17.01,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.97,57.0,H-LC,8.93,169928.0,13751.0,12337.0,0.55,8.80,7.26,16.71,22.0,1.11,1.23,27.78,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,5.18,45.0,H-SC,11.08,125282.0,-15.0,59020.0,1.04,-0.01,47.11,47.09,141.0,-0.00,0.91,56.80,MH,ATH,POWER
32,HAVELLS,2069.16,-0.65,61.0,H-MC,2.31,246152.0,-1579.0,76627.0,-0.31,-0.64,31.13,30.30,92.0,-0.02,1.79,12.57,X40,ATH,ELECTRICAL
56,PGHH,18062.58,-30.02,57.0,H-MC,5.36,202590.0,1770.0,68354.0,-0.73,0.88,33.74,34.92,80.0,0.03,1.47,5.60,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,0.65,41.0,H-LC,9.92,229393.0,-12531.0,75791.0,-0.31,-5.18,33.04,26.15,8.0,-0.17,1.66,8.40,X40,ATH,IT
47,JIOFIN,387.00,-1.31,41.0,H-LC,14.28,218260.0,854.0,52644.0,0.02,0.39,24.12,24.61,48.0,0.02,1.58,55.43,XY24,BTT,FINANCE
61,RELIANCE,1533.00,-14.41,42.0,H-LC,7.65,213829.0,3463.0,25317.0,0.31,1.65,11.84,13.68,37.0,0.14,1.55,18.44,XY25,NTT,REFINERIES
43,INFY,2275.00,-19.24,46.0,H-LC,11.28,315261.0,2373.0,169327.0,-1.24,0.76,53.71,54.87,3.0,0.01,2.29,7.46,X40,BTT,IT
79,TCS,4998.00,-27.08,48.0,H-LC,14.72,291109.0,-54847.0,178712.0,-0.47,-15.85,61.39,35.80,1.0,-0.31,2.11,3.13,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-6.07,51.0,H-LC,2.36,232650.0,-13501.0,92711.0,-1.40,-5.48,39.85,32.18,16.0,-0.15,1.69,28.58,X200,ATH,IT
55,NESTLEIND,1377.00,-8.99,61.0,H-LC,2.92,280121.0,14695.0,42102.0,-0.32,5.54,15.03,21.40,11.0,0.35,2.03,13.02,XY25,NTT,FMCG
45,ITC,452.00,-37.92,53.0,H-LC,3.26,201101.0,963.0,19929.0,1.11,0.48,9.91,10.44,4.0,0.05,1.46,6.07,X40,NTT,FMCG
20,CIPLA,1795.00,-18.41,61.0,H-LC,5.12,217762.0,13262.0,28157.0,1.29,6.49,12.93,20.25,10.0,0.47,1.58,15.78,X40N,BTT,PHARMA
88,VBL,671.64,-10.41,53.0,H-LC,7.46,322732.0,6890.0,105759.0,1.33,2.18,32.77,35.67,5.0,0.07,2.34,16.34,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,17.65,67.0,H-MC,15.71,167968.0,41768.0,4401.0,4.14,33.10,2.62,36.59,108.0,9.49,1.22,49.81,XY24,NTT,HEALTHCARE
64,SAIL,228.00,38.73,58.0,M-MC,7.56,219056.0,-5906.0,173558.0,3.35,-2.63,79.23,74.53,192.0,-0.03,1.59,27.18,XY24,BTT,STEEL
52,MASFIN,398.61,-16.44,58.0,H-SC,13.28,95025.0,-2955.0,24554.0,3.19,-3.02,25.84,22.05,152.0,-0.12,0.69,38.13,XR,ATH,FINANCE
54,NATIONALUM,247.44,-45.32,71.0,H-MC,0.49,109472.0,8968.0,21675.0,3.00,8.92,19.80,30.49,79.0,0.41,0.79,44.22,MH,ATH,METALS
62,REPCOHOME,880.00,-61.25,38.0,H-SC,8.45,143240.0,-61758.0,206996.0,2.89,-30.13,144.51,70.85,134.0,-0.30,1.04,17.68,XY24,NTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-21.65,39.0,H-SC,3.60,110385.0,-17553.0,64686.0,-2.30,-13.72,58.60,36.84,143.0,-0.27,0.80,13.01,X40N,ATH,ELECTRICAL
89,VOLTAS,1918.49,1.30,63.0,H-MC,5.01,212400.0,20658.0,75381.0,-1.71,10.77,35.49,50.08,99.0,0.27,1.54,18.30,XY25,ATH,AC
57,QUESS,986.00,-40.50,41.0,M-SC,42.50,58900.0,-6106.0,157033.0,-1.52,-9.39,266.61,232.18,198.0,-0.04,0.43,1.39,XY24,NTT,MISC
51,LTIM,7230.20,-6.07,51.0,H-LC,2.36,232650.0,-13501.0,92711.0,-1.40,-5.48,39.85,32.18,16.0,-0.15,1.69,28.58,X200,ATH,IT
43,INFY,2275.00,-19.24,46.0,H-LC,11.28,315261.0,2373.0,169327.0,-1.24,0.76,53.71,54.87,3.0,0.01,2.29,7.46,X40,BTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,106.62,69.0,M-SC,6.98,89060.0,-11717.0,11809.0,-0.31,-11.63,13.26,0.09,245.0,-0.99,0.65,17.01,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,85.53,48.0,H-SC,13.84,117342.0,-25137.0,44660.0,0.91,-17.64,38.06,13.70,163.0,-0.56,0.85,38.50,OX40N,NTT,BANKS
41,INDIGOPNTS,1408.0,117.63,54.0,M-SC,3.03,142575.0,-31984.0,32022.0,2.60,-18.32,22.46,0.02,221.0,-1.00,1.03,24.02,OX40N,NTT,PAINTS
48,KANSAINER,340.0,-68.16,55.0,H-SC,3.69,219492.0,-50175.0,86502.0,0.78,-18.61,39.41,13.47,138.0,-0.58,1.59,11.02,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,43.0,H-SC,3.09,113580.0,-30223.0,56972.0,-0.36,-21.02,50.16,18.60,149.0,-0.53,0.82,25.34,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-27.08,48.0,H-LC,14.72,291109.0,-54847.0,178712.0,-0.47,-15.85,61.39,35.80,1.0,-0.31,2.11,3.13,X40,BTT,IT
43,INFY,2275.00,-19.24,46.0,H-LC,11.28,315261.0,2373.0,169327.0,-1.24,0.76,53.71,54.87,3.0,0.01,2.29,7.46,X40,BTT,IT
45,ITC,452.00,-37.92,53.0,H-LC,3.26,201101.0,963.0,19929.0,1.11,0.48,9.91,10.44,4.0,0.05,1.46,6.07,X40,NTT,FMCG
88,VBL,671.64,-10.41,53.0,H-LC,7.46,322732.0,6890.0,105759.0,1.33,2.18,32.77,35.67,5.0,0.07,2.34,16.34,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,248971.0,-12648.0,131855.0,0.75,-4.83,52.96,45.57,7.0,-0.10,1.81,5.94,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,17.65,67.0,H-MC,15.71,167968.0,41768.0,4401.0,4.14,33.10,2.62,36.59,108.0,9.49,1.22,49.81,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186918.0,36005.0,21309.0,0.54,23.86,11.40,37.97,93.0,1.69,1.36,55.20,AR,ATH,AUTO
26,DIXON,18940.15,-29.13,69.0,H-MC,6.82,141816.0,26880.0,9700.0,0.99,23.39,6.84,31.83,81.0,2.77,1.03,47.00,X40N,ATH,IT
27,DMART,5201.00,-7.44,68.0,H-LC,14.33,231123.0,46642.0,23736.0,0.79,25.28,10.27,38.14,38.0,1.97,1.68,38.57,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,17.65,67.0,H-MC,15.71,167968.0,41768.0,4401.0,4.14,33.10,2.62,36.59,108.0,9.49,1.22,49.81,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,13.31,66.0,H-MC,3.23,195520.0,30850.0,37110.0,0.77,18.73,18.98,41.28,89.0,0.83,1.42,28.21,X40N,ATH,AC
89,VOLTAS,1918.49,1.30,63.0,H-MC,5.01,212400.0,20658.0,75381.0,-1.71,10.77,35.49,50.08,99.0,0.27,1.54,18.30,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186918.0,36005.0,21309.0,0.54,23.86,11.40,37.97,93.0,1.69,1.36,55.20,AR,ATH,AUTO
24,DABUR,735.00,-2.58,64.0,H-MC,3.94,211946.0,14918.0,74711.0,-0.21,7.57,35.25,45.49,102.0,0.20,1.54,21.60,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186918.0,36005.0,21309.0,0.54,23.86,11.40,37.97,93.0,1.69,1.36,55.20,AR,ATH,AUTO
54,NATIONALUM,247.44,-45.32,71.0,H-MC,0.49,109472.0,8968.0,21675.0,3.00,8.92,19.80,30.49,79.0,0.41,0.79,44.22,MH,ATH,METALS
90,WHIRLPOOL,2270.00,-39.78,60.0,M-SC,4.86,101782.0,10284.0,68469.0,1.75,11.24,67.27,86.07,223.0,0.15,0.74,47.81,XR,NTT,DURABLES
40,INDIAMART,4850.92,-51.84,44.0,H-SC,12.50,134625.0,11289.0,122468.0,-0.31,9.15,90.97,108.45,119.0,0.09,0.98,33.55,AR,ATH,MISC
91,WIPRO,420.00,-11.98,51.0,M-LC,7.07,154994.0,4049.0,105411.0,-0.37,2.68,68.01,72.51,53.0,0.04,1.12,8.87,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,528.00,2233.16,53.0,H-SC,5.73,62772.0,-18318.0,26986.0,0.94,-22.59,42.99,10.69,156.0,-0.68,0.46,27.22,OX40N,NTT,MISC
65,SAMMAANCAP,326.00,-172.73,70.0,M-SC,7.24,83934.0,-18276.0,111666.0,0.38,-17.88,133.04,91.37,208.0,-0.16,0.61,36.25,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.82,49.0,H-SC,16.22,90498.0,-10465.0,104996.0,2.08,-10.37,116.02,93.63,148.0,-0.10,0.66,34.79,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.45,93792.0,-19757.0,59811.0,2.45,-17.40,63.77,35.27,268.0,-0.33,0.68,129.56,XR,NTT,HOTELS
52,MASFIN,398.61,-16.44,58.0,H-SC,13.28,95025.0,-2955.0,24554.0,3.19,-3.02,25.84,22.05,152.0,-0.12,0.69,38.13,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,69.0,L-SC,33.45,93792.0,-19757.0,59811.0,2.45,-17.40,63.77,35.27,268.0,-0.33,0.68,129.56,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186918.0,36005.0,21309.0,0.54,23.86,11.40,37.97,93.0,1.69,1.36,55.20,AR,ATH,AUTO
60,RELAXO,1176.00,-31.07,63.0,H-SC,6.31,64000.0,-51538.0,86528.0,2.75,-44.61,135.20,30.28,136.0,-0.60,0.46,25.82,X40N,NTT,FOOTWEAR
14,BLUESTARCO,2326.38,13.31,66.0,H-MC,3.23,195520.0,30850.0,37110.0,0.77,18.73,18.98,41.28,89.0,0.83,1.42,28.21,X40N,ATH,AC
27,DMART,5201.00,-7.44,68.0,H-LC,14.33,231123.0,46642.0,23736.0,0.79,25.28,10.27,38.14,38.0,1.97,1.68,38.57,X40N,NTT,FMCG


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.22
1,25,42.45
2,50,72.09


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.47
SC    36.49
MC    26.04
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.84
X40      15.09
X40N     13.94
XY25     11.22
AR        9.18
XR        8.81
OX40N     7.21
X5K       2.25
MH        1.70
X200      1.69
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.48
H-MC    23.16
H-SC    21.85
M-SC    13.08
M-LC     6.96
M-MC     2.56
L-SC     1.56
L-LC     1.03
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,1.20,30.99
IT,12.99,-15.36,72.57
FINANCE,7.17,-22.02,67.02
BANKS,7.15,-15.80,64.97
PAINTS,5.85,-14.38,39.97
MISC,5.72,-16.79,83.74
ELECTRICAL,5.57,-7.48,66.71
AUTO,4.49,-9.34,56.90
AC,3.67,4.25,28.17


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2802201.0,23
AR,1216244.0,10
XR,1078593.0,13
X40,888652.0,10
X40N,511872.0,11
OX40N,464498.0,10
XY25,434492.0,8
SR,196775.0,2
X5K,127940.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2766416.0,27
M-SC,1443430.0,17
H-LC,1407376.0,18
H-MC,1186758.0,18
M-LC,419214.0,5
M-MC,324635.0,2
L-SC,245203.0,3
L-MC,59177.0,1
L-LC,42464.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,929567.0,6
M-SC,XY24,858992.0,7
H-SC,AR,664955.0,5
H-LC,X40,593230.0,5
H-SC,XR,522206.0,6
H-MC,XY24,460786.0,5
H-LC,AR,369752.0,2
M-MC,XY24,324635.0,2
H-MC,X40,219640.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
